In [1]:
import os

import pandas as pd

# Prepare app data

This notebook reads TEOTIL3 results files from the OsloMod project and aggregates them for use in a simple Streamlit app.

In [2]:
# Period of interest
st_yr, end_yr = 2017, 2019

# Define vassdragsområder for Oslofjord
#  - Inner Oslofjord vassoms 005 to 009
#  - Outer Oslofjord vassoms 001 to 004 and 010 to 014
#  - Whole Oslofjord vassoms 001 to 014
vassom_dict = {
    "Indre Oslofjord": list(range(5, 10)),
    "Ytre Oslofjord": list(range(1, 5)) + list(range(10, 18)),
    "Hele Oslofjord": list(range(1, 18)),
}

oslomod_scen_dir = r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios"

In [3]:
result_csv = os.path.join(
    oslomod_scen_dir, f"teotil3_oslomod_results_{st_yr}-{end_yr}.csv"
)
df = pd.read_csv(result_csv)

main_catches = [f"{i:03d}." for i in vassom_dict["Hele Oslofjord"]]
df = df.query("regine in @main_catches")

val_cols = [
    col for col in df.columns if col.startswith("accum_") and col.endswith("_kg")
]
id_cols = ["scenario", "year", "regine"]
df = df[id_cols + val_cols]
df = df.melt(id_vars=id_cols)

df[["stat", "src", "par", "unit"]] = df["variable"].str.split("_", expand=True)
df["value"] = df["value"] / 1000
df["par"] = df["par"]
del df["variable"], df["unit"], df["stat"]

src_dict = {
    "aquaculture": "Akvakultur",
    "agriculture": "Jordbruk",
    "large-wastewater": "Kommunalt avløp",
    "spredt": "Spredt avløp",
    "industry": "Industri",
    "urban": "Bebygd",
    "agriculture-background": "Bakgrunn",
    "glacier": "Bakgrunn",
    "upland": "Bakgrunn",
    "wood": "Bakgrunn",
    "lake": "Bakgrunn",
}
df["src"] = df["src"].replace(src_dict)
df["par"] = df["par"].str.upper()
df["regine"] = df["regine"].str[:-1]
df = df.groupby(["scenario", "year", "regine", "src", "par"]).sum().reset_index()
df = df.rename(
    columns={
        "scenario": "Scenario",
        "regine": "Område",
        "src": "Kilde",
        "par": "Parameter",
        "value": "Verdi (tonn)",
    }
)

df = df.groupby(["Scenario", "Område", "Kilde", "Parameter"]).mean().reset_index()
df["Verdi (tonn)"] = df["Verdi (tonn)"]  # .round(3)
del df["year"]

df = df[df["Verdi (tonn)"] > 0]

# Totals for Oslofjord
for area, vassom_list in vassom_dict.items():
    catches = [f"{i:03d}" for i in vassom_list]
    area_df = (
        df.query("Område in @catches")
        .groupby(["Scenario", "Kilde", "Parameter"])
        .sum(numeric_only=True)
        .reset_index()
    )
    area_df["Område"] = area

    df = pd.concat([df, area_df], axis="rows")

df.to_csv("../data/results_summary.csv", index=False)
df.head()

,Scenario,Område,Kilde,Parameter,Verdi (tonn)
7,Baseline,001,Bakgrunn,DIN,109.079145
8,Baseline,001,Bakgrunn,SS,485.593558
9,Baseline,001,Bakgrunn,TDP,1.138774
10,Baseline,001,Bakgrunn,TOC,7407.268113
11,Baseline,001,Bakgrunn,TON,222.815918
